# origin of country 와 viewpoint 분석

In [1]:
import pandas as pd
import numpy as np
import pandas_profiling

# 데이터 시각화
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

# 가설검정
from scipy import stats

# 경고문구 무시
import warnings
warnings.filterwarnings("ignore")
# 자연어 처리
# import spacy

## 데이터 정보 확인

In [2]:
df=pd.read_csv('../data/1_original_useDf_0711.csv')

In [3]:
# null값 확인
df.isnull().sum()

Num                         0
Title                       0
Series or Movie             0
COUNTRY                   206
Release Date               65
GENRE                     370
VALUE                       0
Genre_all                 118
Tags                        9
Languages                 149
Hidden Gem Score          152
Country Availability        5
Runtime                     0
Director                  713
Writer                    514
Actors                    138
View Rating               708
IMDb Score                152
Rotten Tomatoes Score    1385
Metacritic Score         1510
Awards Received          1490
Awards Nominated For     1090
Boxoffice                1665
Netflix Release Date        0
Production House         1492
Summary                     2
IMDb Votes                152
dtype: int64

In [4]:
df.describe().round()

,Num,VALUE,Hidden Gem Score,IMDb Score,Rotten Tomatoes Score,Metacritic Score,Awards Received,Awards Nominated For,IMDb Votes
count,2960.0,2960.0,2808.0,2808.0,1575.0,1450.0,1470.0,1870.0,2808.0
mean,1480.0,634.0,5.0,6.0,54.0,54.0,11.0,21.0,94402.0
std,855.0,2202.0,2.0,1.0,27.0,17.0,24.0,41.0,190158.0
min,0.0,1.0,1.0,2.0,0.0,5.0,1.0,1.0,5.0
25%,740.0,22.0,3.0,6.0,30.0,41.0,2.0,3.0,1759.0
50%,1480.0,80.0,4.0,6.0,54.0,53.0,4.0,7.0,17586.0
75%,2219.0,321.0,7.0,7.0,78.0,66.0,10.0,19.0,107313.0
max,2959.0,44867.0,10.0,10.0,100.0,100.0,300.0,386.0,2308981.0


In [5]:
df['country_all']=df['COUNTRY']+','+ df['Country Availability']

In [6]:
revise_country = df['country_all'].str.split(',') #리스트화

In [7]:
# 중복값 제거
for i in range(int(len(revise_country))):
     revise_country[i]= np.unique(revise_country[i]).tolist()

In [8]:
revise_country = pd.DataFrame(revise_country)

In [9]:
revise_country

,country_all
0,"[Argentina, Australia, Belgium, Brazil, Canada..."
1,"[Argentina, Australia, Belgium, Brazil, Canada..."
2,"[Argentina, Australia, Belgium, Brazil, Canada..."
3,"[Argentina, Australia, Belgium, Brazil, Canada..."
4,"[Argentina, Australia, Belgium, Brazil, Canada..."
...,...
2955,"[Argentina, Australia, Belgium, Brazil, Colomb..."
2956,"[Canada, Czech Republic, Hungary, Romania, Sou..."
2957,"[Germany, India, United States]"
2958,"[Argentina, Australia, Brazil, Canada, Colombi..."


In [10]:
df['country_all'] = revise_country

In [14]:
rows = len(df['country_all'])
df['showingOn_count'] = 0
for i in range(rows):
    df.showingOn_count[i]=len(str(df['country_all'].iloc[i]).split(','))

In [15]:
df

,Num,Title,Series or Movie,COUNTRY,Release Date,GENRE,VALUE,Genre_all,Tags,Languages,...,Metacritic Score,Awards Received,Awards Nominated For,Boxoffice,Netflix Release Date,Production House,Summary,IMDb Votes,country_all,showingOn_count
0,0,thequeensgambit,TV,United States,2020.10.23,Drama,44867,"Biography, Drama, History","TV Dramas,US TV Shows,TV Shows Based on Books,...","English, German, French",...,91.0,96.0,97.0,"$56,441,711",2020.10.23,"Pathé Pictures, Scott Rudin Productions","In a 1950s orphanage, a young girl reveals an ...",104495.0,"[Argentina, Australia, Belgium, Brazil, Canada...",36
1,1,365days,Movie,Poland,2020.6.7,Romance,42149,"Drama, Romance","Polish Movies,Polish Dramas,Dramas,Romantic Dr...","Polish, English, Italian",...,NaN,NaN,NaN,NaN,2020.4.2,NaN,A fiery executive in a spiritless relationship...,50125.0,"[Argentina, Australia, Belgium, Brazil, Canada...",36
2,2,emilyinparis,TV,United States,2020.10.2,Drama,27138,"Comedy, Drama, Romance","Comedy Programmes,Drama Programmes,Romantic TV...","French, English",...,NaN,1.0,3.0,NaN,2020.10.2,NaN,"After landing her dream job in Paris, Chicago ...",45000.0,"[Argentina, Australia, Belgium, Brazil, Canada...",36
3,3,lucifer,TV,United States,2016.1.25,Superhero,25729,"Horror, Thriller","TV Dramas,Police TV Shows,TV Sci-Fi & Fantasy,...",English,...,NaN,NaN,NaN,NaN,2017.7.24,NaN,"Bored with being the Lord of Hell, the devil r...",57.0,"[Argentina, Australia, Belgium, Brazil, Canada...",35
4,4,theumbrellaacademy,TV,United States,2019.2.15,Superhero,23480,"Action, Adventure, Comedy, Drama, Fantasy, Sci-Fi","TV Sci-Fi & Fantasy,US TV Shows,TV Action & Ad...",English,...,NaN,7.0,28.0,NaN,2019.2.15,NaN,"Reunited by their fathers death, estranged sib...",159677.0,"[Argentina, Australia, Belgium, Brazil, Canada...",36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2955,2955,thebournesupremacy,Movie,United States,2004.7.23,Action,1,"Action, Mystery, Thriller","Action Thrillers,Spy Action & Adventure,Action...","English, Russian, German, Italian",...,73.0,5.0,22.0,"$176,241,941",2015.4.14,"Universal Pictures, Hypnotic, Kennedy/Marshall","Flushed from his refuge by an assassin, Jason ...",435887.0,"[Argentina, Australia, Belgium, Brazil, Colomb...",31
2956,2956,justfriends,Movie,United States,2005.11.23,Romance,1,"Comedy, Romance","Romantic Comedies,Comedies,Romantic Films,Roma...",English,...,47.0,NaN,3.0,"$32,619,671",2015.4.14,"Cinerenta, Benderspink, Just Friends Productio...","Years after high school, a once-portly nice gu...",106951.0,"[Canada, Czech Republic, Hungary, Romania, Sou...",7
2957,2957,child44,Movie,United States,2015.3.15,Thriller,1,"Crime, Drama, History, Mystery, Thriller","Dramas based on a book,Dramas,Thrillers,Crime ...","English, Russian",...,41.0,NaN,2.0,"$1,224,330",2016.12.4,Scott Free Productions,"In 1950s Russia, a former war hero investigate...",66251.0,"[Germany, India, United States]",3
2958,2958,draculauntold,Movie,United States,2014.10.3,Horror,1,"Action, Drama, Fantasy, Horror, War","Horror Movies,Sci-Fi & Fantasy,Action & Advent...","English, Turkish",...,40.0,3.0,5.0,"$56,280,355",2016.3.28,Michael De Luca,A medieval warrior and ruler must become a vam...,180432.0,"[Argentina, Australia, Brazil, Canada, Colombi...",30


In [12]:
df.to_csv('3_original_useDf_0714.csv.csv',encoding='utf-8-sig')

In [16]:
dfdf= df[['Num','Title','VALUE','Series or Movie','COUNTRY','GENRE','country_all','showingOn_count']]

In [17]:
dfdf.to_csv('3.1_original_useDf_0714.csv.csv',encoding='utf-8-sig')